In [ ]:
### import csv
import os
from typing import List, Dict, Optional


# ---------------- Student Model ---------------- #

class Student:
    def __init__(self, sid: str, name: str, marks: List[int]):
        self.id = sid
        self.name = name
        self.marks = marks
        self.average = self.calculate_average()
        self.grade = self.calculate_grade()
        self.rank: int | None = None

    def calculate_average(self) -> float:
        return sum(self.marks) / len(self.marks)

    def calculate_grade(self) -> str:
        avg = self.average
        if avg >= 90:
            return "A"
        elif avg >= 75:
            return "B"
        elif avg >= 60:
            return "C"
        elif avg >= 40:
            return "D"
        else:
            return "F"


# ---------------- Storage Layer ---------------- #

class StudentStorage:
    FILE_NAME = "students.csv"
    FIELDNAMES = ["id", "name", "marks", "average", "grade"]

    @classmethod
    def load_students(cls) -> List[Student]:
        students = []
        if not os.path.exists(cls.FILE_NAME):
            return students

        with open(cls.FILE_NAME, "r", newline="", encoding="utf-8") as file:
            reader = csv.DictReader(file)
            for row in reader:
                marks = list(map(int, row["marks"].split(",")))
                student = Student(row["id"], row["name"], marks)
                students.append(student)
        return students

    @classmethod
    def save_students(cls, students: List[Student]) -> None:
        with open(cls.FILE_NAME, "w", newline="", encoding="utf-8") as file:
            writer = csv.DictWriter(file, fieldnames=cls.FIELDNAMES)
            writer.writeheader()
            for s in students:
                writer.writerow({
                    "id": s.id,
                    "name": s.name,
                    "marks": ",".join(map(str, s.marks)),
                    "average": f"{s.average:.2f}",
                    "grade": s.grade
                })


# ---------------- Manager / Controller ---------------- #

class StudentManager:
    def __init__(self):
        self.students: List[Student] = StudentStorage.load_students()

    # ---------- Helpers ---------- #

    def prompt_non_empty(self, prompt: str) -> str:
        while True:
            value = input(prompt).strip()
            if value:
                return value
            print("⚠️ Value cannot be empty.")

    def parse_marks(self, raw: str) -> List[int]:
        parts = [p.strip() for p in raw.split(",")]
        if any(p == "" for p in parts):
            raise ValueError("Empty value found.")
        marks = list(map(int, parts))
        if any(m < 0 or m > 100 for m in marks):
            raise ValueError("Marks must be between 0 and 100.")
        return marks

    def find_by_id(self, sid: str) -> Optional[Student]:
        for s in self.students:
            if s.id == sid:
                return s
        return None

    # ---------- CRUD ---------- #

    def add_student(self):
        sid = self.prompt_non_empty("Enter ID: ")
        if self.find_by_id(sid):
            print("⚠️ ID already exists")
            return

        name = self.prompt_non_empty("Enter Name: ")
        while True:
            try:
                marks = self.parse_marks(
                    self.prompt_non_empty("Enter marks (comma separated): ")
                )
                break
            except ValueError as e:
                print(e)

        self.students.append(Student(sid, name, marks))
        StudentStorage.save_students(self.students)
        print("✅ Student added")

    def update_student(self):
        sid = self.prompt_non_empty("Enter ID to update: ")
        student = self.find_by_id(sid)
        if not student:
            print("❌ Student not found")
            return

        student.name = self.prompt_non_empty("Enter new name: ")
        student.marks = self.parse_marks(
            self.prompt_non_empty("Enter new marks: ")
        )
        student.average = student.calculate_average()
        student.grade = student.calculate_grade()

        StudentStorage.save_students(self.students)
        print("✏️ Student updated")

    def delete_student(self):
        sid = self.prompt_non_empty("Enter ID to delete: ")
        student = self.find_by_id(sid)
        if not student:
            print("❌ Student not found")
            return

        self.students.remove(student)
        StudentStorage.save_students(self.students)
        print("🗑️ Student deleted")

    def display_students(self):
        if not self.students:
            print("No students found.")
            return

        self.students.sort(key=lambda s: s.average, reverse=True)
        for i, s in enumerate(self.students, start=1):
            s.rank = i

        print("\nID | Name | Avg | Grade | Rank")
        print("-" * 40)
        for s in self.students:
            print(f"{s.id} | {s.name} | {s.average:.2f} | {s.grade} | {s.rank}")

    # ---------- Menu ---------- #

    def run(self):
        while True:
            print("""
1. Add Student
2. Update Student
3. Delete Student
4. Display Students
5. Exit
            """)
            choice = input("Enter choice: ").strip()

            if choice == "1":
                self.add_student()
            elif choice == "2":
                self.update_student()
            elif choice == "3":
                self.delete_student()
            elif choice == "4":
                self.display_students()
            elif choice == "5":
                print("👋 Exiting...")
                break
            else:
                print("⚠️ Invalid choice")


# ---------------- Entry Point ---------------- #

if __name__ == "__main__":
    StudentManager().run()


         

      

   


1. Add Student
2. Update Student
3. Delete Student
4. Display Students
5. Exit
            


Enter choice:  1
Enter ID:  501
Enter Name:  Bhimisetty Manasa
Enter marks (comma separated):  90,98,87,87,77


✅ Student added

1. Add Student
2. Update Student
3. Delete Student
4. Display Students
5. Exit
            


Enter choice:  2
Enter ID to update:  Bee Reddy Arvind


❌ Student not found

1. Add Student
2. Update Student
3. Delete Student
4. Display Students
5. Exit
            


...


ID | Name | Avg | Grade | Rank
----------------------------------------
501 | Bhimisetty Manasa | 87.80 | B | 1
503 | Pranay | 85.80 | B | 2
502 | Bee Reddy Arvind | 80.00 | B | 3

1. Add Student
2. Update Student
3. Delete Student
4. Display Students
5. Exit
            


Enter choice:  2
Enter ID to update:  503
Enter new name:  Bhimisetty Bala Pranay
Enter new marks:  90,78,65,90,56


✏️ Student updated

1. Add Student
2. Update Student
3. Delete Student
4. Display Students
5. Exit
            


Enter choice:  4



ID | Name | Avg | Grade | Rank
----------------------------------------
501 | Bhimisetty Manasa | 87.80 | B | 1
502 | Bee Reddy Arvind | 80.00 | B | 2
503 | Bhimisetty Bala Pranay | 75.80 | B | 3

1. Add Student
2. Update Student
3. Delete Student
4. Display Students
5. Exit
            
